In [3]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
iris=load_iris()
svm=SVC(gamma='auto')

In [7]:
clf = GridSearchCV(svm,{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,return_train_score = False)
clf.fit(iris.data,iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00159965, 0.00060039, 0.00279689, 0.00259671, 0.0025619 ,
        0.0025826 ]),
 'std_fit_time': array([0.00120158, 0.00049023, 0.00074343, 0.00049238, 0.00049008,
        0.00046966]),
 'mean_score_time': array([0.00140057, 0.0004024 , 0.00180011, 0.00139537, 0.00183358,
        0.00159879]),
 'std_score_time': array([0.00048951, 0.00049285, 0.00075153, 0.00048811, 0.00037604,
        0.00082593]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],
 'split0_test_score': ar

In [9]:
import pandas as pd
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001600,0.001202,0.001401,0.000490,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000600,0.000490,0.000402,0.000493,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.002797,0.000743,0.001800,0.000752,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.002597,0.000492,0.001395,0.000488,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.002562,0.000490,0.001834,0.000376,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.002583,0.000470,0.001599,0.000826,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [11]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

Use RandomizedSearchCV when we have too many parameters to try and training time takes longer.

In [13]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm,{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,return_train_score = False,n_iter = 2)
rs.fit(iris.data , iris.target)
pd.DataFrame(rs.cv_results_)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001195,0.000750,0.001000,0.000634,rbf,10,"{'kernel': 'rbf', 'C': 10}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.002045,0.000946,0.000965,0.000596,linear,10,"{'kernel': 'linear', 'C': 10}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,2


In [15]:
rs.best_params_

{'kernel': 'rbf', 'C': 10}

To select which model works best 

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter =100000)
rf=RandomForestClassifier()

In [31]:
model_params= {
    'svm': {
        'model': svm,
        'params' : {
            'C': [1,10,20],
            'kernel' : ['rbf','linear']
        }
    },
    'random forest' : {
        'model': rf,
        'params' : {
            'n_estimators':[1,5,10]
        }
    },
    'logistic regression' : {
        'model' : lr ,
        'params' : {
            'C':[1,5,10]
        }
    }
}

In [33]:
scores = []
for model_name,mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score = False)
    clf.fit(iris.data,iris.target)
    scores.append({
    'model' : model_name,
    'best_score' : clf.best_score_,
    'best_params' : clf.best_params_})
df = pd.DataFrame(scores)
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random forest,0.966667,{'n_estimators': 5}
2,logistic regression,0.973333,{'C': 1}


Based on the above I can conclude that svm model with parameters C=1 and kernel = rbf will give best results for solving my problem of iris classification